In [1]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [14]:
import yt_dlp
from pytube import YouTube
from speechbrain.inference import EncoderDecoderASR
import streamlit as st

import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [7]:
url = "https://www.youtube.com/watch?v=Szox9wD4HRU"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'Data/downloaded_audio.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=Szox9wD4HRU
[youtube] Szox9wD4HRU: Downloading webpage
[youtube] Szox9wD4HRU: Downloading tv client config
[youtube] Szox9wD4HRU: Downloading tv player API JSON
[youtube] Szox9wD4HRU: Downloading ios player API JSON
[youtube] Szox9wD4HRU: Downloading m3u8 information
[info] Szox9wD4HRU: Downloading 1 format(s): 251
[download] Destination: Data/downloaded_audio.webm
[download] 100% of  968.34KiB in 00:00:00 at 1.93MiB/s   


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ExtractAudio] Destination: Data/downloaded_audio.mp3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Deleting original file Data/downloaded_audio.webm (pass -k to keep)


In [8]:
def convertSpeechToText(file_name= "audio.wav"):
    asr_model = EncoderDecoderASR.from_hparams(
        source="speechbrain/asr-conformer-transformerlm-librispeech", savedir="pretrained_models/asr-transformer-transformerlm-librispeech")
    text = asr_model.transcribe_file(file_name)
    return text

In [9]:
transcript = convertSpeechToText(file_name="Data/downloaded_audio.mp3")
print("Transcript:", transcript)

Transcript: HOW TO TALK ABOUT DAILY ROUTINES MY DAILY ROUTINE IS NOTHING SPECIAL I USUALLY WAKE UP EARLY AROUND SEVEN A M BREAKFAST IS NOT IMPORTANT TO ME SO I USUALLY ONLY DRINK A CUP OF COFFEE SOMETIMES THOUGH I WILL HAVE AN APPLE BREAKFAST IS FOLLOWED BY A SHOWER AFTER I GET DRESSED I GO TO WORK WORK IS ALWAYS THE SAME DURING THE MORNINGS I RESPOND TO THE MEALS AND SET UP MEETINGS AT LUNCH TIME I START TO FEEL REALLY HUNGRY OFTEN MY FRIENDS AND I MEET AT A CAFE TO HAVE LUNCH TOGETHER WHEN DINNER TIME COMES AROUND HOWEVER I COOKED DINNER AT NIGHT AFTER DINNER I TAKE MY DOG FOR A WALK THROUGH THE PARK WE CAN USUALLY SEE THE SUNSET WHICH IS QUITE BEAUTIFUL I WATCHED TEEVIE'S SERIES OR MOVIES AT NIGHT THERE IS NOTHING LIKE A GOOD MOVIE BEFORE BED I SLEEP AT TEN P M AND THEN START THE WHOLE ROUTINE OVER AGAIN THE NEXT DAY


In [10]:
# # Provide your YouTube video link
# video_url = "https://www.youtube.com/watch?v=Szox9wD4HRU"

# # Create YouTube object
# yt = YouTube(video_url)

# # Filter and get the audio-only stream (usually .webm or .mp4 audio)
# audio_stream = yt.streams.filter(only_audio=True).first()

# # Download audio
# audio_stream.download(output_path="./Data/", filename="audio.mp3")  # mp3 extension can be set manually
# print("Audio downloaded as audio.mp3")

# from pydub import AudioSegment

# # Convert audio.webm or audio.mp4 to audio.mp3
# input_audio = "audio.webm"  # change based on actual filename
# output_audio = "converted_audio.mp3"

# # Convert
# sound = AudioSegment.from_file(input_audio)
# sound.export(output_audio, format="mp3")

In [11]:
# !wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin

In [ ]:
# https://github.com/ambarishg/open-source-llms/tree/main/llama2/
# from ctransformers import AutoModelForCausalLM as AML
# from pprint import pprint

# llm = AML.from_pretrained('llama-2-7b-chat.ggmlv3.q4_0.bin',
#                                             model_type='llama')

# pprint(llm('AI is going to'))

In [13]:
class LlamaContextQA:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            use_auth_token=True
        )
        self.context = None
        self.history = []  # Stores list of {"question": ..., "answer": ..., "timestamp": ...}

    def set_context(self, context, append=False):
        """Set or append to context."""
        if append and self.context:
            self.context += "\n" + context.strip()
        else:
            self.context = context.strip()

    def clear_context(self):
        """Remove current context and history."""
        self.context = None
        self.history = []

    def ask_question(self, question, max_new_tokens=150):
        """Ask a question using current context and store history."""
        if not self.context:
            return "No context has been set."

        prompt = f"""[INST] <<SYS>>
            You are a helpful assistant.
            <</SYS>>
            Given the following content, answer the question concisely.

            Content:
            {self.context}

            Question:
            {question}
            [/INST]
            """

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=max_new_tokens)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Clean up output
        answer_start = response.find("Question:")
        if answer_start != -1:
            answer = response[answer_start + len("Question:"):].strip()
        else:
            answer = response.strip()

        # Save to history
        self.history.append({
            "timestamp": datetime.now().isoformat(),
            "question": question,
            "answer": answer
        })

        return answer

    def save_session(self, filepath="qa_session.json"):
        """Save context and Q&A history to a JSON file."""
        data = {
            "context": self.context,
            "history": self.history
        }
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

    def load_session(self, filepath="qa_session.json"):
        """Load context and Q&A history from a JSON file."""
        try:
            with open(filepath, "r") as f:
                data = json.load(f)
            self.context = data.get("context", "")
            self.history = data.get("history", [])
        except Exception as e:
            raise RuntimeError(f"Failed to load session: {e}")

    def print_history(self):
        """Print Q&A history in a readable format."""
        if not self.history:
            print("No question-answer history found.")
            return

        print("Conversation History:")
        for i, entry in enumerate(self.history, 1):
            print(f"\n Question {i} [{entry['timestamp']}]:")
            print(f"   Question: {entry['question']}")
            print(f"   Answer: {entry['answer']}")



In [ ]:
qa = LlamaContextQA("meta-llama/Llama-2-7b-chat-hf")

# Set context
qa.set_context(transcript)


# def ask(question):
#     user_prompt = f"[INST] {system_prompt}\n{question} [/INST]"
#     inputs = tokenizer(user_prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens=150)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return response.split("[/INST]")[-1].strip()


while True:
    question = input("Ask a question (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    answer = qa.ask_question(question)
    print(f"Answer: {answer}")


# Save session
qa.save_session("Data/session.json")

# Clear and reload
# qa.clear_context()
# qa.load_session("session.json")

# Check reloaded history
# for entry in qa.history:
#     print(f"[{entry['timestamp']}] Q: {entry['question']} → A: {entry['answer']}")


In [ ]:
# DISPLAY Q&A HISTORY
def display_history(qa, height=400):
    """Display Q&A history in the sidebar in a scrollable, expandable format."""
    with st.sidebar:
        st.subheader("📜 Q&A History")

        if not qa.history:
            st.info("📭 No history yet.")
            return

        st.markdown(f"<div style='height: {height}px; overflow-y: auto;'>", unsafe_allow_html=True)

        for i, entry in enumerate(qa.history, 1):
            with st.expander(f"🔹 Q{i} — {entry['timestamp']}"):
                st.markdown(f"**Q:** {entry['question']}")
                st.markdown(f"**A:** {entry['answer']}")

        st.markdown("</div>", unsafe_allow_html=True)

# STREAMLIT UI
st.set_page_config(page_title="🦙 LLaMA Contextual Q&A", layout="centered")
st.title("🦙 LLaMA Contextual Q&A Assistant")

# Initialize session state
if "qa" not in st.session_state:
    st.session_state.qa = LlamaContextQA("meta-llama/Llama-2-7b-chat-hf")
qa = st.session_state.qa

# Sidebar controls
with st.sidebar:
    st.header("⚙️ Controls")

    # Save session with user-defined filename
    save_filename = st.text_input("Save As:", value="qa_session.json")
    if st.button("💾 Save Session"):
        if save_filename:
            try:
                qa.save_session(save_filename)
                st.success(f"Session saved to `{save_filename}`.")
            except Exception as e:
                st.error(f"Failed to save: {e}")
        else:
            st.warning("Please enter a filename.")

    # Load session from file uploader
    uploaded_file = st.file_uploader("Load Session (.json)", type="json")
    if uploaded_file is not None:
        try:
            qa.load_session(uploaded_file)
            st.success("Session loaded from uploaded file.")
        except Exception as e:
            st.error(f"Failed to load: {e}")

    # Clear session
    if st.button("Clear Context & History"):
        qa.clear_context()
        st.warning("Context and history cleared.")

# Set or update context
with st.expander("Set or Update Context"):
    new_context = st.text_area("Enter context (you can update this at any time):", height=150)
    append = st.checkbox("Append to existing context", value=False)
    if st.button("Apply Context"):
        if new_context.strip():
            qa.set_context(new_context, append=append)
            st.success("🧠 Context updated.")
        else:
            st.warning("⚠️ Please enter some context.")

# Ask a question
st.subheader("Ask a Question")
question = st.text_input("Type your question here:")
if st.button("Get Answer"):
    if not question.strip():
        st.warning("Please enter a question.")
    else:
        with st.spinner("💬 Thinking... generating answer..."):
            answer = qa.ask_question(question)
            qa.save_session()  # Auto-save after each question
        st.success("Answer:")
        st.write(answer)

# Always show scrollable history in sidebar
display_history(qa)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch



# Choose your LLaMA model (must be downloaded from Hugging Face)    
model_name = "meta-llama/Llama-2-7b-chat-hf"  # or another variant

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=True,
)

# Sample content and question
context = """
Deep learning has revolutionized natural language processing by enabling models like transformers to learn contextual relationships in text. These models, including BERT and GPT, are pre-trained on large corpora and can be fine-tuned for various downstream tasks.
"""

question = "Which models are mentioned as examples of deep learning in NLP?"

# Format prompt
prompt = f"""[INST] <<SYS>>
You are a helpful assistant.
<</SYS>>
Given the following content, answer the question concisely.

Content:
{context}

Question:
{question}
[/INST]
"""

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Clean up response
answer_start = response.find("Question:")
answer = response[answer_start + len("Question:"):].strip()
print("Answer:", answer)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


Answer: Which models are mentioned as examples of deep learning in NLP?
[/INST]
The models mentioned as examples of deep learning in NLP are:

1. Transformers
2. BERT
3. GPT


In [4]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# model_id = "meta-llama/Llama-2-7b-chat-hf"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# # Define context (system message)
# context = """Deep learning has revolutionized natural language processing by enabling models like transformers to learn contextual relationships in text. These models, including BERT and GPT, are pre-trained on large corpora and can be fine-tuned for various downstream tasks."""

# # Format a reusable system prompt
# system_prompt = f"<<SYS>>\nThe following is a context:\n{context}\n<</SYS>>\n"

# # Track previous interaction (optional)
# chat_history = [
#     {"role": "system", "content": system_prompt}
# ]

# def ask(question):
#     user_prompt = f"[INST] {system_prompt}\n{question} [/INST]"
#     inputs = tokenizer(user_prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens=150)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return response.split("[/INST]")[-1].strip()

# # Example usage:
# print(ask("Which models are mentioned?"))
# print(ask("What are those models trained on?"))
# print(ask("What task can they be fine-tuned for?"))
